# Determinação de atributos das quadras com densidade construtiva a partir de dados do IPTU, série total

In [1]:
import geopandas as gpd
import pandas as pd
import rasterio
import numpy as np

## Carregando e pré-processando dados do IPTU

In [2]:
# Padrões de nomenclatura de usos

usos = {
    "apartamento em condomínio":"apartamento",
    "cortiço (habitação coletiva subnormal)":"cortiço",
    "escritório/consultório em condomínio (unidade autônoma)":"escritório ou consultório",
    "loja em edifício em condomínio (unidade autônoma)":"loja em edifício em condomínio",
    "residência coletiva, exclusive cortiço (mais de uma residência no lote)":"residência coletiva (mais de uma residência no lote), exclusive cortiço",
    "posto de serviço (combustíveis)":"posto de serviço",
    "estacionamento e garagem, não em condomínio":"garagem (exclusive em prédio em condomínio)",
    "garagem (unidade autônoma) de prédio de garagens":"garagem, em prédio de garagens",
    "garagem (unidade autônoma) em edifício em condomínio de escritórios, consultórios ou misto":"garagem em edifício de escritórios, consultórios ou misto",
    "garagem (unidade autônoma) em edifício em condomínio de uso exclusivamente residencial":"garagem em edifício de uso exclusivamente residencial",
    "outras edificações de uso coletivo, com utilização múltipla":"outras edificações do tipo (uso coletivo), com utilização múltipla",
    "outras edificações de uso comercial, com utilização múltipla":"outras edificações do tipo (uso comércio), com utilização múltipla",
    "outras edificações de uso de serviço, com utilização múltipla":"outras edificações do tipo (uso serviço), com utilização múltipla",
    "outras edificações de uso especial, com utilização múltipla":"outras edificações do tipo (uso especial), com utilização múltipla",
    "prédio com uso exclusivamente residencial, não em condomínio":"prédio de apartamento, não em condomínio, de uso exclusivamente residencial",
    "prédio com uso misto, predominância de uso não residencial, não em condomínio":"prédio de escritório, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios) com ou sem loja (predominância comercial)",
    "prédio com uso misto, predominância de uso residencial, não em condomínio":"prédio de apartamento, não em condomínio, de uso misto (apartamentos e escritórios e/ou consultórios), com ou sem loja (predominância residencial)",
    "prédio de escritório ou consultório, com ou sem lojas, não em condomínio":"prédio de escritório ou consultório, não em condomínio, com ou sem lojas",
    "flat de uso comercial (semelhante a hotel)":"flat, não residencial",
    "flat residencial em condomínio":"flat, residencial"
}

In [3]:
df_iptu = pd.read_csv('IPTU/IPTU_2020.zip',\
             compression='zip',\
             encoding='iso-8859-9',\
             sep=';',\
             decimal=',',\
             error_bad_lines=False)

In [4]:
# Obter o SQ (Setor e Quadra)
df_iptu['setor'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:3]
df_iptu['quadra'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[3:6]

# Calculando chave 'SQ'
df_iptu['sq'] = \
df_iptu['setor'] + df_iptu['quadra']

# Normalizando o lote condominial
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] != '00', 'lote'] = '0000'
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'lote'] = \
df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'NUMERO DO CONTRIBUINTE'].str[6:10]

# Individualizando a testada e pavimentos por lote (SQLC)
df_iptu['sqlc'] = df_iptu['setor'] + \
df_iptu['quadra'] + df_iptu['lote'] + \
df_iptu['NUMERO DO CONDOMINIO'].str[0:2]

df_iptu['AREA CONSTRUIDA'] = pd.to_numeric(df_iptu['AREA CONSTRUIDA'], errors ='coerce')
df_iptu['AREA OCUPADA'] = pd.to_numeric(df_iptu['AREA OCUPADA'], errors ='coerce')
df_iptu['AREA DO TERRENO'] = pd.to_numeric(df_iptu['AREA DO TERRENO'], errors ='coerce')
df_iptu['FRACAO IDEAL'] = pd.to_numeric(df_iptu['FRACAO IDEAL'], errors ='coerce')
df_iptu['QUANTIDADE DE PAVIMENTOS'] = pd.to_numeric(df_iptu['QUANTIDADE DE PAVIMENTOS'], errors ='coerce')
df_iptu['TESTADA PARA CALCULO'] = pd.to_numeric(df_iptu['TESTADA PARA CALCULO'], errors ='coerce')

# Adequando variáveis à fração ideal
df_iptu['area_terreno'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA DO TERRENO']
df_iptu['area_ocupada'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA OCUPADA']

# Adequar os usos ao padrão estabelecido
df_iptu.loc[df_iptu['TIPO DE USO DO IMOVEL']\
            .isin(usos),\
            'TIPO DE USO DO IMOVEL'] = \
df_iptu[df_iptu['TIPO DE USO DO IMOVEL']\
             .isin(usos)]['TIPO DE USO DO IMOVEL']\
.apply(lambda x: usos[x])

## Carregando e pré-processando quadras

In [5]:
# Carregando geometria das quadras fiscais
gdf_quadras = gpd.GeoDataFrame.from_file(f'zip://GIS/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF')

In [6]:
gdf_quadras['sq'] = gdf_quadras.qd_setor + gdf_quadras.qd_fiscal

In [7]:
gdf_quadras = gdf_quadras.loc[gdf_quadras.qd_tipo == 'F'][['sq', 'geometry']].dissolve(by='sq')

In [8]:
# gdf_quadras.geometry = gdf_quadras.representative_point()

## Agregando e espacializando dado de densidade contruída

In [9]:
# Agregar por SQ e ANO DO EXERCICIO
iptu_agg = df_iptu[['sq', \
                    'AREA CONSTRUIDA',
                    'area_terreno']]\
.groupby('sq')\
.agg({'AREA CONSTRUIDA': 'sum',
     'area_terreno': 'sum'})\
.rename(columns = {'AREA CONSTRUIDA': 'area_construida'})

In [10]:
# iptu_agg['densidade'] = round(iptu_agg['area_construida'] / iptu_agg['area_terreno'], 2)

In [11]:
# iptu_agg['densidade']

In [12]:
# iptu_agg['densidade'] = iptu_agg['densidade'].round(decimals = 2).astype('float16')

In [16]:
iptu_agg.rename(columns={'area_construida':'2020_area_construida',\
                        'area_terreno':'2020_area_terreno'})

,2020_area_construida,2020_area_terreno
sq,,
001003,29926,21529.9343
001004,20715,14196.1760
001005,17263,9484.0000
001006,7686,3453.9722
001007,26067,14844.4308
...,...,...
310116,1313,1273.0000
310117,22601,17716.1264
310118,25215,17934.5581


In [17]:
# Gerando GeoDataFrame
gdf_iptu_agg = gdf_quadras.merge(iptu_agg, on='sq', how='right')

In [18]:
# gdf_iptu_agg.loc['171198']

In [19]:
# df_iptu.loc[df_iptu.sq == '171198']

In [20]:
# gdf_iptu_agg.drop(columns=['area_construida', 'area_terreno'])

In [21]:
del df_iptu

In [22]:
del gdf_iptu_agg

In [23]:
# gdf_iptu_agg.to_file('resultados/seila.gpkg', driver='GPKG')

## Gera raster densidade construtiva para cada ano

In [25]:
from pathlib import Path
import os
import numpy as np

In [26]:
densidades = []

In [27]:
for path in Path('IPTU').rglob('*.zip'):
    print(path.name)
    
    # 1. Carregar dados do arquivo
    # OBS.: O IPTU de 2016 está separado por ',' e não por ';'
    if path.name == "IPTU_2016.zip":
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=',',\
                     decimal=',',\
                     error_bad_lines=False)
    else:
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=';',\
                     decimal=',',\
                     error_bad_lines=False)
        
    # Obter o SQ (Setor e Quadra)
    df_iptu['setor'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:3]
    df_iptu['quadra'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[3:6]

    # Calculando chave 'SQ'
    df_iptu['sq'] = \
    df_iptu['setor'] + df_iptu['quadra']

    # Normalizando o lote condominial
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] != '00', 'lote'] = '0000'
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'lote'] = \
    df_iptu.loc[df_iptu['NUMERO DO CONDOMINIO'].str[0:2] == '00', 'NUMERO DO CONTRIBUINTE'].str[6:10]

    # Individualizando a testada e pavimentos por lote (SQLC)
    df_iptu['sqlc'] = df_iptu['setor'] + \
    df_iptu['quadra'] + df_iptu['lote'] + \
    df_iptu['NUMERO DO CONDOMINIO'].str[0:2]

    df_iptu['AREA CONSTRUIDA'] = pd.to_numeric(df_iptu['AREA CONSTRUIDA'], errors ='coerce')
    df_iptu['AREA OCUPADA'] = pd.to_numeric(df_iptu['AREA OCUPADA'], errors ='coerce')
    df_iptu['AREA DO TERRENO'] = pd.to_numeric(df_iptu['AREA DO TERRENO'], errors ='coerce')
    df_iptu['FRACAO IDEAL'] = pd.to_numeric(df_iptu['FRACAO IDEAL'], errors ='coerce')
    df_iptu['QUANTIDADE DE PAVIMENTOS'] = pd.to_numeric(df_iptu['QUANTIDADE DE PAVIMENTOS'], errors ='coerce')
    df_iptu['TESTADA PARA CALCULO'] = pd.to_numeric(df_iptu['TESTADA PARA CALCULO'], errors ='coerce')

    # Adequando variáveis à fração ideal
    df_iptu['area_terreno'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA DO TERRENO']
    df_iptu['area_ocupada'] = df_iptu['FRACAO IDEAL'] * df_iptu['AREA OCUPADA']

    # Adequar os usos ao padrão estabelecido
    df_iptu.loc[df_iptu['TIPO DE USO DO IMOVEL']\
                .isin(usos),\
                'TIPO DE USO DO IMOVEL'] = \
    df_iptu[df_iptu['TIPO DE USO DO IMOVEL']\
                 .isin(usos)]['TIPO DE USO DO IMOVEL']\
    .apply(lambda x: usos[x])

    # Agregar por SQ e ANO DO EXERCICIO
    iptu_agg = df_iptu[['sq', \
                        'AREA CONSTRUIDA',
                        'area_terreno',
                        'area_ocupada',
                        'QUANTIDADE DE PAVIMENTOS',
                        'TESTADA PARA CALCULO']]\
    
    # Agregar por SQ e ANO DO EXERCICIO
    iptu_agg = df_iptu[['sq', \
                        'AREA CONSTRUIDA',
                        'area_terreno']]\
    .groupby('sq')\
    .agg({'AREA CONSTRUIDA': 'sum',
         'area_terreno': 'sum'})\
    .rename(columns = {'AREA CONSTRUIDA': 'area_construida'})
    
    
    # Removendo colunas sobressalentes
    densidades.append(iptu_agg.rename(columns={'area_construida':f'{path.name[-8:-4]}_area_construida',\
                                               'area_terreno':f'{path.name[-8:-4]}_area_terreno'}))
    
    # Gerando GeoDataFrame
    #gdf_iptu_agg = gdf_quadras.merge(iptu_agg, on='sq', how='right')

IPTU_2010.zip


b'Skipping line 2445507: expected 35 fields, saw 43\n'
/home/fernando/miniconda3/envs/geoplot/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


IPTU_1996.zip


/home/fernando/miniconda3/envs/geoplot/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


IPTU_2007.zip


...

b'Skipping line 2440049: expected 35 fields, saw 43\n'


IPTU_2003.zip


b'Skipping line 2288886: expected 35 fields, saw 43\n'


IPTU_2012.zip


b'Skipping line 2526350: expected 35 fields, saw 43\n'


In [29]:
df_densidades = \
pd.concat(densidades, axis=1).reindex(columns=sorted(pd.concat(densidades, axis=1).columns))

In [31]:
df_densidades

,1995_area_construida,1995_area_terreno,1996_area_construida,1996_area_terreno,1997_area_construida,1997_area_terreno,1998_area_construida,1998_area_terreno,1999_area_construida,1999_area_terreno,...,2016_area_construida,2016_area_terreno,2017_area_construida,2017_area_terreno,2018_area_construida,2018_area_terreno,2019_area_construida,2019_area_terreno,2020_area_construida,2020_area_terreno
sq,,,,,,,,,,,,,,,,,,,,,
001003,37464.0,21448.9773,39154.0,21448.9773,39120.0,21456.9773,39120.0,21456.9773,39120.0,21477.9773,...,40035.0,21529.9343,40035.0,21529.9343,40035.0,21529.9343,31102.0,21529.9343,29926.0,21529.9343
001004,20616.0,14173.1760,20870.0,14173.1760,21621.0,14173.1760,21645.0,14173.1760,21645.0,14173.1760,...,20701.0,14196.1760,20701.0,14196.1760,20715.0,14196.1760,20715.0,14196.1760,20715.0,14196.1760
001005,14488.0,9480.4600,14488.0,9480.4600,16268.0,9480.4600,16368.0,9480.4600,16368.0,9480.4600,...,17103.0,9484.0000,17103.0,9484.0000,17263.0,9484.0000,17263.0,9484.0000,17263.0,9484.0000
001006,6378.0,3474.9722,5909.0,3337.9722,7728.0,3465.9722,7728.0,3465.9722,7728.0,3465.9722,...,7686.0,3453.9722,7686.0,3453.9722,7686.0,3453.9722,7686.0,3453.9722,7686.0,3453.9722
001007,22929.0,14939.1304,22890.0,14773.1304,23922.0,14946.1304,23922.0,14946.1304,23922.0,14951.1304,...,26042.0,14952.1304,26067.0,14844.4308,26067.0,14844.4308,26067.0,14844.4308,26067.0,14844.4308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277055,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2606.0,3526.0000
300115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,104240.0000
304156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20891.0,26472.0000


In [32]:
df_densidades.replace([np.inf, -np.inf], np.nan, inplace=True)

In [67]:
# df_densidades = df_densidades.interpolate(method='linear', limit_direction='forward', axis=1)

In [33]:
# df_densidades.loc['169197']

In [34]:
# df_densidades.loc['169197']

In [35]:
# df_densidades.loc['171198']

In [36]:
# Gerando GeoDataFrame
gdf_iptu_agg = gdf_quadras.merge(df_densidades, on='sq', how='left')

In [72]:
# gdf_iptu_agg.loc['169197']

In [37]:
gdf_iptu_agg.to_file('resultados/densidade_construtiva/IPTU_quadras_area_contruida.gpkg', driver='GPKG')

In [31]:
# np.arange(1995, 2021)

In [32]:
# gdf_iptu_agg.replace([np.inf, -np.inf], np.nan, inplace=True)

In [33]:
# gdf_iptu_agg.iloc[45142].drop('geometry').std()

In [ ]:
# gdf_iptu_agg.iloc[45142].drop('geometry')

In [ ]:
# gdf_iptu_agg.iloc[45142].drop('geometry').ewm(com=0.3).mean()